In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
def load_data(filename):
    stripper = str.strip
    res = []
    with open(filename, 'r') as f:
        for line in f:
            question, label  = map(stripper,line.rstrip('\n').split(",,,", 1))
            res.append((label, question))            
    return res


In [3]:
training_data_path = "./LabelledData.txt"
train_data = load_data(training_data_path)

In [4]:
data = pd.DataFrame(train_data, columns =['labels','questions'])

In [5]:
data['labels'].unique()

array(['unknown', 'what', 'when', 'who', 'affirmation'], dtype=object)

In [6]:
from sklearn.utils import shuffle
data = shuffle(data)

In [7]:
Y = data[['labels']]

In [9]:
Y.sum()

In [10]:
X = data[['questions']]

In [19]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize(text):   
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = text.split(" ")
    return tokens

# Tokenizing the text elements
vectorizer = TfidfVectorizer(analyzer='word',tokenizer=tokenize,lowercase=True,max_features =500)
corpus_data_features = vectorizer.fit_transform(X.questions.tolist() )

#Convering the document term matrix to numpy nd array
corpus_data_features_nd = (corpus_data_features.toarray())
print (corpus_data_features_nd.shape)

(1483, 500)


In [12]:
X.head()

,questions
601,what is `` the bear of beers '' ?
703,where are the tropical rain forest distributio...
1252,when was richard nixon born ?
1272,who created the monster in mary shelley 's nov...
351,what is the speed of the mississippi river ?


In [20]:
from sklearn.svm import LinearSVC

# Model Declaration
my_model = LinearSVC()

# Fit model with train and test data
my_model = my_model.fit(X=corpus_data_features_nd, y=Y.values.ravel())
test_pred = my_model.predict(corpus_data_features_nd)

C:\Users\Amit\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
from sklearn.metrics import accuracy_score

accuracy_score(test_pred,Y)

0.98786244099797704

In [22]:
test_pred

array(['what', 'unknown', 'when', ..., 'unknown', 'affirmation', 'what'], dtype=object)

In [23]:
response = vectorizer.transform(['What time will the train arrive'])

#Convering the document term matrix to numpy nd array
pred_nd = (response.toarray())

In [24]:
my_model.predict(pred_nd)

array(['when'], dtype=object)

In [25]:
import pickle
with open("x_result.pkl", 'wb') as handle:
    pickle.dump( vectorizer, handle)
#load the content
tfidf = pickle.load(open("x_result.pkl", "rb" ) )

In [76]:
response = tfidf.transform(['What time will the train arrive'])

#Convering the document term matrix to numpy nd array
pred_nd = (response.toarray())

In [26]:
filename = 'model.sav'
import pickle
pickle.dump(my_model, open(filename, 'wb'))